## **Denoising Autoencoder**

NeuralA Denoising Autoencoder (DAE) is a type of autoencoder designed to learn robust representations by training the model to reconstruct the original, clean input from a noisy version. The model adds noise to the input during training to force the autoencoder to focus on the most important features of the data.

**Imports**

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import numpy as np

**Data Loading**

In [ ]:
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Minimal Preprocessing**

In [ ]:
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)
X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

**Denoising Autoencoder Model**

In [ ]:
class DenoisingAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(DenoisingAutoencoder, self).__init__()
        self.encoder = nn.Linear(input_dim, hidden_dim)
        self.decoder = nn.Linear(hidden_dim, input_dim)
    
    def forward(self, x):
        encoded = torch.relu(self.encoder(x))
        decoded = torch.sigmoid(self.decoder(encoded))
        return decoded

**Train the Model with Noise**

In [ ]:
def add_noise(x, noise_factor=0.2):
    noise = torch.randn_like(x) * noise_factor
    return torch.clamp(x + noise, 0, 1)

model = DenoisingAutoencoder(input_dim=X_train.shape[1], hidden_dim=8)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

for epoch in range(50):
    optimizer.zero_grad()
    noisy_input = add_noise(torch.tensor(X_train, dtype=torch.float32))
    y_pred = model(noisy_input)
    loss = loss_fn(y_pred, torch.tensor(X_train, dtype=torch.float32))
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/50], Loss: {loss.item():.4f}")